# Clasificacion de texto usando algoritmos de DataLab


Para esto se utiliza como ejemplo la data del rubro alimentos de Mercado Público:



*   df_alimentos: https://drive.google.com/file/d/1IMtwgQ1c3Rq1f9iVO_5UTjFUSFmZQjlF/view

## 1. Cargar archivos y dependencias


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df_alimentos= pd.read_csv('/content/drive/MyDrive/df_alimentos.csv')

In [ ]:
df_alimentos.sample(100)

,id,buyer_specs,quantity,Rubro,Clase
466600,7233467,Betarragas\r\n,15.0,"Alimentos, bebidas y tabaco",Verduras
622557,5878127,YOGURT DE LITRO SOPROLE DE FRUTILLA - VAINILLA,10.0,"Alimentos, bebidas y tabaco",Postres y decoraciones
319020,4122310,CILANTRO.,1.0,"Alimentos, bebidas y tabaco",Verduras
552135,8620176,OREGANO,5.0,"Alimentos, bebidas y tabaco","Hierbas, especias y extractos"
562471,8756730,PATE EMBUTIDO DE 150 GRS,50.0,"Alimentos, bebidas y tabaco","Salsas, condimentos y productos para untar"
...,...,...,...,...,...
754828,11407354,(Cód. interno 010101000122) Sopa: Crema de cha...,4.0,"Alimentos, bebidas y tabaco",Sopas y estofados
140351,1861748,"QUESILLO, SIMILAR COLUN O SUPERIOR, EMBUTIDOS ...",70.0,"Alimentos, bebidas y tabaco",Quesos
702188,8307833,AGUA MINERAL SIN GAS 1 1/2 LITROS,15.0,"Alimentos, bebidas y tabaco",Bebidas no alcohólicas
599506,5398076,SALSA DE SOYA 1 LT,12.0,"Alimentos, bebidas y tabaco","Salmuera, salsas y aceitunas"


Dependencias: SE RECOMIENDA UTILIZAR GPU

In [ ]:
# Package installation (hidden on docs.cleanlab.ai).
# If running on Colab, may want to use GPU (select: Runtime > Change runtime type > Hardware accelerator > GPU)
# Package versions we used:scikit-learn==1.2.0 sentence-transformers==2.2.2

dependencies = ["cleanlab", "sklearn", "sentence_transformers", "datasets"]

# Supress outputs that may appear if tensorflow happens to be improperly installed:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # disable parallelism to avoid deadlocks with huggingface

if "google.colab" in str(get_ipython()):  # Check if it's running in Google Colab
    %pip install cleanlab==v2.4.0
    cmd = ' '.join([dep for dep in dependencies if dep != "cleanlab"])
    %pip install $cmd
else:
    missing_dependencies = []
    for dependency in dependencies:
        try:
            __import__(dependency)
        except ImportError:
            missing_dependencies.append(dependency)

    if len(missing_dependencies) > 0:
        print("Missing required dependencies:")
        print(*missing_dependencies, sep=", ")
        print("\nPlease install them before running the rest of this notebook.")

In [ ]:
import re
import string
import pandas as pd
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer

from cleanlab import Datalab
import numpy as np

In [ ]:
# This cell is hidden from docs.cleanlab.ai

import random
import numpy as np

#pd.set_option("display.max_colwidth", 10)

SEED = 123456  # for reproducibility
np.random.seed(SEED)
random.seed(SEED)

## 1. Definir modelo de clasificación y obtención de probabilidades de predicción.

Veamos el desbalance de clases

In [ ]:
class_counts = df_alimentos['Clase'].value_counts()
class_porcentaje = df_alimentos['Clase'].value_counts(normalize=True) * 100

# Crear un nuevo DataFrame con las columnas 'Clase', 'Cantidad' y 'Porcentaje'
df_summary = pd.DataFrame({'Clase': class_counts.index,
                           'Cantidad': class_counts.values,
                           'Porcentaje': class_porcentaje.values})

# Imprimir el nuevo DataFrame
df_summary


,Clase,Cantidad,Porcentaje
0,Verduras,188316,23.757475
1,Frutas,72318,9.123458
2,Bebidas no alcohólicas,65804,8.301668
3,"Pan, galletas y pastelitos dulces",64979,8.197588
4,Carnes y aves de corral,43341,5.467792
5,Productos de leche y mantequilla,39684,5.006434
6,Café y té,29943,3.777534
7,Carnes procesadas y preparadas,27404,3.457220
8,"Productos de chocolates, azúcares y edulcorantes",19361,2.442535
9,Postres y decoraciones,17540,2.212802


Filtremos las clases que tengan muy pocos datos

In [ ]:
filtro = list(df_summary.iloc[-1:].Clase)

In [ ]:
df_filtrado = df_alimentos[~df_alimentos['Clase'].isin(filtro)]

In [ ]:
#df_muestra = df_filtrado.groupby('Clase').apply(lambda x: x.sample(900)).drop(columns='Clase')
#df_muestra = df_muestra.reset_index()
df_muestra = df_filtrado.copy()

### Pre-procesamiento de Texto

Cargamos stopwords

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopwords_es = set(stopwords.words('spanish'))
# Palabras adicionales para conservar
palabras_a_conservar = {'te'}  # Agrega otras palabras que deseas conservar

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Funciones auxiliares para generar una descripción más limpia

In [ ]:
import re

def limpiar(texto):
    puntuación = r'[,;.:¡!¿?@#$%&[\](){}<>~=+\-*/|\\_^`"\']'
    patrones_eliminar = r'\b(nº|n°|kg|lt|cm|ml|mm|lp|km|gr|lp|litro|kilo|metro|centimetro|metros|centimetros|kilogramos|kilos|litros|market|gramos|gramo|grs|kgs|cms|lts|merkat| o|similar|formato|marca|referencia|acuenta|misol|botella|traverso|tipo|rm|cc|pq|k|unidad|lemon stone|lata|x|aal|unidades|paquete|sabor|tradicion|region|v|suministro|individual|aproximado|mensual|anual|colun|soprole|mckay|triton|nestle|tipo|comestible|similar)\b'  # Patrones a eliminar
    palabras_eliminar = ['adquisi', 'servicio', 'contrat', 'articulo', 'accesorio', 'cotizacion',
                        'insumo', 'equipamiento', 'suministro', 'construccion', 'mejoramiento', 'mantencion', 'instalacion', 'compra', 'chica',
                        'arriendo', 'reparacion', 'adjudicada', 'adulto', 'equipo', 'hospital', 'segun',
                        'item', 'bases', 'especificaciones', 'tecnicas', 'adjunt', 'refrigerador', 'marca', 'modelo',
                        'unidad', 'bolsa', 'litro', 'anexo', 'similar', 'linea', 'capacidad', 'descripcion',
                        'equivalente', 'presentacion', 'tipo', 'desechable', 'diametro', 'punta', 'compatible',
                        'catalogo', 'referencia', 'canal', 'caja', 'volumen', 'acuerdo', 'licitacion', 'largo', 'administrat', 'debe', 'valor', 'indicad', 'total', 'neto', 'oferta', 'region', 'indicar', 'nombre', 'fabricante', 'caracter', 'comuna', 'establecimient', 'varios', 'provincia', 'talla', 'entrega', 'superior', 'dias', 'habil', 'vencimiento', 'requiere', 'provision', 'sugiere', 'detalle', 'incluido', 'ejecucion', 'garantia', 'minima', 'cumplen', 'anos',
                        '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'rigen', 'proyecto', 'media', 'implementacion', 'municipal', 'convenio', 'marco', 'publico', 'reposicion', 'cesfam']

    # Signos de puntuación
    texto = re.sub(puntuación, ' ', texto)

    # Dígitos [0-9]
    texto = re.sub('\d', ' ', texto)

    # Eliminar patrones específicos
    texto = re.sub(patrones_eliminar, ' ', texto)

    # Eliminar palabras específicas
    for palabra in palabras_eliminar:
        texto = re.sub(r'\b{}\b'.format(palabra), ' ', texto)

    return texto


def normalizar(texto):
    # todo a minúsculas
    texto = texto.lower()

    # tildes y diacríticas
    texto = re.sub('á', 'a', texto)
    texto = re.sub('é', 'e', texto)
    texto = re.sub('í', 'i', texto)
    texto = re.sub('ó', 'o', texto)
    texto = re.sub('ú', 'u', texto)
    texto = re.sub('ü', 'u', texto)
    texto = re.sub('ñ', 'n', texto)

    return texto

Pre procesamiento Parte 1:

In [ ]:
df_muestraa = df_muestra.copy()
df_muestraa['buyer_specs_2'] = df_muestraa['buyer_specs'].apply(normalizar)
df_muestraa['buyer_specs_2'] = df_muestraa['buyer_specs_2'].apply(limpiar)
df_muestraa

,id,buyer_specs,quantity,Rubro,Clase,buyer_specs_2
0,232428,JUGO SURTIDO DE 7 GRS.,10.0,"Alimentos, bebidas y tabaco",Bebidas no alcohólicas,jugo surtido de
1,232429,POROTOS,1.0,"Alimentos, bebidas y tabaco",Legumbres,porotos
2,232430,ARROZ GRADO 2,2.0,"Alimentos, bebidas y tabaco",Cereales,arroz grado
3,232431,HARINA BLANCA DE 5 KILOS,2.0,"Alimentos, bebidas y tabaco",Cereales,harina blanca de
4,232432,SEMOLA,1.0,"Alimentos, bebidas y tabaco",Cereales procesados,semola
...,...,...,...,...,...,...
792655,10670297,TOMATES,7.0,"Alimentos, bebidas y tabaco",Verduras,tomates
792656,10667385,FRUTILLAS,20.0,"Alimentos, bebidas y tabaco",Frutas,frutillas
792657,10667386,PIÑA,20.0,"Alimentos, bebidas y tabaco",Frutas,pina
792658,10670295,PAPAS,30.0,"Alimentos, bebidas y tabaco",Verduras,papas


Pre-procesamiento Parte 2:

In [ ]:
# Tokenización de las descripciones
df_muestraa['buyer_specs_tokens'] = df_muestraa['buyer_specs_2'].apply(nltk.word_tokenize)
# Aplicar eliminación de stopwords con excepción de 'te'
df_muestraa['buyer_specs_tokens'] = df_muestraa['buyer_specs_tokens'].apply(lambda tokens: [token for token in tokens if token.lower() not in stopwords_es or token.lower() in palabras_a_conservar])
# Convertir lista de tokens a cadena de texto
df_muestraa['buyer_specs_processed'] = df_muestraa['buyer_specs_tokens'].apply(lambda tokens: ' '.join(tokens))
df_muestraa['buyer_specs_processed'] = df_muestraa['buyer_specs_processed'].apply(lambda text: re.sub(r'[\(\)\d\W_]', ' ', text))
df_muestraa.sample(10)

,id,buyer_specs,quantity,Rubro,Clase,buyer_specs_2,buyer_specs_tokens,buyer_specs_processed
388070,9305085,"PISCO SOUR, UNIDAD",34.00,"Alimentos, bebidas y tabaco",Bebidas alcohólicas,pisco sour,"[pisco, sour]",pisco sour
8,207961,(1430157 )PURE INSTANTANEO MAGGI BOLSA 2K UNID...,40.00,"Alimentos, bebidas y tabaco",Verduras,pure instantaneo maggi bolsa ...,"[pure, instantaneo, maggi, bolsa]",pure instantaneo maggi bolsa
233473,4568076,"Endulzante Sucralosa 500 tabletas, Iansa o End...",24.00,"Alimentos, bebidas y tabaco","Productos de chocolates, azúcares y edulcorantes",endulzante sucralosa tabletas iansa endo...,"[endulzante, sucralosa, tabletas, iansa, endol...",endulzante sucralosa tabletas iansa endolce ca...
420377,9768585,leche en polvo,4.00,"Alimentos, bebidas y tabaco",Productos de leche y mantequilla,leche en polvo,"[leche, polvo]",leche polvo
560654,8731096,Zanahorias,3.20,"Alimentos, bebidas y tabaco",Verduras,zanahorias,[zanahorias],zanahorias
451242,6986054,CANELA EN POLVO BOLSA,10.00,"Alimentos, bebidas y tabaco",Carnes procesadas y preparadas,canela en polvo bolsa,"[canela, polvo, bolsa]",canela polvo bolsa
11894,79830,POSTA ROSADA BEEF,14.78,"Alimentos, bebidas y tabaco",Carnes y aves de corral,posta rosada beef,"[posta, rosada, beef]",posta rosada beef
322509,4182903,QUINOA ALMIFRUT BOLSA 1K UNIDAD XVI REGION,6.00,"Alimentos, bebidas y tabaco",Pasta o tallarines naturales,quinoa almifrut bolsa xvi,"[quinoa, almifrut, bolsa, xvi]",quinoa almifrut bolsa xvi
478846,7422644,"AAL-0141\r\nCEBOLLAS, VARIEDADES, UNIDAD\r\n",80.00,"Alimentos, bebidas y tabaco",Verduras,\r\ncebollas variedades \r\n,"[cebollas, variedades]",cebollas variedades
27204,371593,SOPA EN SOBRE MAGGI VARIEDADES,6.00,"Alimentos, bebidas y tabaco",Sopas y estofados,sopa en sobre maggi variedades,"[sopa, maggi, variedades]",sopa maggi variedades


### Procesamiento: Texto a Tensores con word embeddings

In [ ]:
data = df_muestraa.copy()
data.head()

,id,buyer_specs,quantity,Rubro,Clase,buyer_specs_2,buyer_specs_tokens,buyer_specs_processed
0,4467119,LÁMINAS PARA TERMOLAMINAR,4.0,"Equipos, accesorios y suministros de oficina",Suministros para plastificado,laminas termolaminar,"['laminas', 'termolaminar']",laminas termolaminar
1,4467121,PACK DE 4 TINTAS 544 COLORES (EPSON),3.0,"Equipos, accesorios y suministros de oficina","Suministros para impresora, fax y fotocopiadora",pack de tintas colores,"['pack', 'tintas', 'colores']",pack tintas colores
2,4467123,DESTACADORES PASTEL,15.0,"Equipos, accesorios y suministros de oficina",Instrumentos de escritura,destacadores pastel,"['destacadores', 'pastel']",destacadores pastel
3,4467124,LAPICES DE CERA 12 COLORES GIOTTO PAX,15.0,"Equipos, accesorios y suministros de oficina",Instrumentos de escritura,lapices de cera colores giotto pax,"['lapices', 'cera', 'colores', 'giotto', 'pax']",lapices cera colores giotto pax
4,4467128,LAPICES DE COLORES ARTEL LARGOS 12 COLORES,15.0,"Equipos, accesorios y suministros de oficina",Instrumentos de escritura,lapices de colores artel largos ...,"['lapices', 'colores', 'artel', 'largos', 'col...",lapices colores artel largos colores


In [ ]:

raw_texts, labels = data["buyer_specs_processed"].values, data["Clase"].values
num_classes = len(set(labels))

print(f"This dataset has {num_classes} classes.")
print(f"Classes: {set(labels)}")

This dataset has 30 classes.
Classes: {'Empaquetadoras', 'Máquinas de correo', 'Multicopistas, fotocopiadoras, fax y multifuncionales', 'Máquinas encuadernadoras y plastificadoras', 'Máquinas calculadoras y accesorios', 'Suministros para el manejo de efectivo', 'Suministros de correo', 'Sistemas de planificación', 'Máquinas y accesorios para registro de hora y asistencia en la oficina', 'Máquinas para procesamiento de papel y accesorios', 'Fuibles o fusores y accesorios', 'Máquinas para endosar y extender cheques', 'Suministros de dibujo', 'Etiquetadoras', 'Suministros para impresora, fax y fotocopiadora', 'Máquinas de escribir y accesorios', 'Agendas y accesorios', 'Medios de corrección', 'Suministros de sujeción', 'Suministros para plastificado', 'Accesorios de máquinas de oficina', 'Suministros de escritorio', 'Suministros de máquinas de encuadernar', 'Accesorios para impresoras, fotocopiadoras y aparatos de fax', 'Tableros o pizarras', 'Máquinas clasificadoras', 'Instrumentos de es

Este paso puede demorar desde 3 min hasta horas, dependiendo la dimensión de los datos:

In [ ]:
transformer = SentenceTransformer('google/electra-small-discriminator')
text_embeddings = transformer.encode(raw_texts)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/google_electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Modelo de clasificación
Aca se puede usar uno a elección. De momento el de mejor resultados es Balanced Random Forest

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(class_weight="balanced")
pred_probs = cross_val_predict(model, text_embeddings, labels, method="predict_proba")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.para

## 4. Aplicando CleanLab para encontrar anomalías

Se guarda la data en un diccionario

In [ ]:
data_dict = {"texts": raw_texts, "labels": labels}

Se aplica DataLab ingresando las probabilidades pronosticadas y las text embeddings obtenidas anteriormente.

In [ ]:
lab = Datalab(data_dict, label_name="labels")
lab.find_issues(pred_probs=pred_probs, features=text_embeddings)

Reporte:

In [ ]:
#Data pre procesada
lab.report()

### Etiquetas erróneas

CleanLab detecta etiquetas con problemas, a través de 2 indicadores, 'is_label_issue' y 'label_score'. Label_score indica un valor entre 0 y 1, donde un valor cercano a 0 indica una mayor probabilidad de error en la etiqueta original.

In [ ]:
label_issues = lab.get_issues("label")
label_issues

Armamos un nuevo dataframe incluyendo las métricas en el original.

In [ ]:
df_reetiquetas = label_issues.copy()
df_reetiquetas['id'] = df_muestra.id
df_reetiquetas['descripcion'] = df_muestra.buyer_specs
df_reetiquetas['Clase'] = df_muestra.Clase
df_reetiquetas = df_reetiquetas[['id','descripcion','Clase','predicted_label','is_label_issue','label_score']]
df_reetiquetas

### Outlier issues

According to the report, our dataset contains some outliers.
We can see which examples are outliers (and a numeric quality score quantifying how typical each example appears to be) via `get_issues`. We sort the resulting DataFrame by cleanlab's outlier quality score to see the most severe outliers in our dataset.

In [ ]:
outlier_issues = lab.get_issues("outlier")
outlier_issues.sort_values("outlier_score")
df_final = pd.merge(df_reetiquetas,outlier_issues,left_index=True,right_index=True)
df_final.sort_values('outlier_score')

We see that cleanlab has identified entries in this dataset that do not appear to be proper customer requests. Outliers in this dataset appear to be out-of-scope customer requests and other nonsensical text which does not make sense for intent classification. Carefully consider whether such outliers may detrimentally affect your data modeling, and consider removing them from the dataset if so.

#Dataframe procesado

In [ ]:
df_final

In [ ]:
df_final.query('is_label_issue == True & label_score <=0.3').sort_values('label_score',ascending=False).head(60)

In [ ]:

# Crear una copia del DataFrame original
df_final_corrected = df_final.copy()

# Aplicar la condición y actualizar los valores en la nueva columna
mask = (df_final['is_label_issue'] == True) & (df_final['label_score'] <= 0.3)
df_final_corrected.loc[mask, 'Clase_nueva'] = df_final.loc[mask, 'predicted_label']
df_final_corrected.loc[~mask, 'Clase_nueva'] = df_final.loc[~mask, 'Clase']

df_final_corrected


In [ ]:
df_final_corrected[['id','descripcion','Clase_nueva']].sample(100)